In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install pycaret
import imblearn
from imblearn.over_sampling import SMOTE
from pycaret.classification import *
#from pycaret.regression import *

Using TensorFlow backend.


In [2]:
print(imblearn.__version__)

0.6.2


# Importing the data - only use *.MLReady.csv files!!!

In [18]:
df=pd.read_csv("D:\FM_Acq_Perf_FMAC_EPOCH3_ML.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,id,origChannel,origIntRate,origUPB,origDate,firstPmtDate,origLTV,numBorrowers,...,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct
0,1265166,1265166,100495721320,1,4.375,310000,2013-11-01,2014-01-01,64.0,1.0,...,4.16,4.255,0.045673,2.65,0.025,-0.215,0.120,0.005747,-0.051683,0.028202
1,1265167,1265167,102361386857,1,4.625,142000,2013-11-01,2014-01-01,69.0,1.0,...,4.16,4.255,0.045673,2.65,0.275,-0.465,0.370,0.063218,-0.111779,0.086957
2,1265168,1265168,102516953457,1,5.000,124000,2013-11-01,2014-01-01,80.0,1.0,...,4.16,4.255,0.045673,2.65,0.650,-0.840,0.745,0.149425,-0.201923,0.175088
3,1265169,1265169,103711862089,3,4.490,170000,2013-11-01,2014-01-01,54.0,2.0,...,4.16,4.255,0.045673,2.65,0.140,-0.330,0.235,0.032184,-0.079327,0.055229
4,1265170,1265170,104417831957,3,4.750,100000,2013-11-01,2014-01-01,32.0,1.0,...,4.16,4.255,0.045673,2.65,0.400,-0.590,0.495,0.091954,-0.141827,0.116334


In [19]:
df.iloc[0]

Unnamed: 0                         1265166
Unnamed: 0.1                       1265166
id                            100495721320
origChannel                              1
origIntRate                          4.375
origUPB                             310000
origDate                        2013-11-01
firstPmtDate                    2014-01-01
origLTV                                 64
numBorrowers                             1
origDebtIncRatio                        50
borrCreditScore                        751
loanPurp                                 1
zipCode                                220
pMIperct                                 0
mortInsType                              0
bestCreditScore                        751
worstCreditScore                       751
avgCreditScore                         751
bankNumber                              54
stateNumber                             47
rptPeriod                       2016-05-01
currIntRate                          4.375
currUPB    

In [20]:
# Target variable is zeroBalCode
df.zeroBalCode.value_counts()

1.0     33440
9.0      1010
2.0       401
6.0       182
16.0      170
3.0       123
15.0       62
Name: zeroBalCode, dtype: int64

 Target is ~ 5.5% of total population

## Remove columns not involved in 'Foreclosure or Not' query

In [21]:
df.drop(['Unnamed: 0','Unnamed: 0.1','id','firstPmtDate','rptPeriod','currIntRate','currUPB','loanAge','monMatur',\
         'zeroBalDate','lastPdInstDate','forecloDate','dispDate','forecloCost','propRepCost','recovCosts',\
         'miscCost','holdTaxCost','saleProceed','credEnhProceed','repurchProceed','otherForecloProceed',\
         'nonIntUPB','prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator',\
         'deliqGood','deliqBad','deliqMax'],1,inplace=True)

In [22]:
#df.info()

## Engineer originDate into Date features

In [23]:
df['origDate'] = pd.to_datetime(df['origDate'])

In [24]:
df['origYear'] = df['origDate'].dt.year
df['origMonth'] = df['origDate'].dt.month

In [25]:
df.drop(['origDate'],1,inplace=True)

In [26]:
df[['origYear','origMonth']].head()

,origYear,origMonth
0,2013,11
1,2013,11
2,2013,11
3,2013,11
4,2013,11


# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [27]:
df.zeroBalCode.unique()

array([ 1.,  9., 16.,  3.,  2.,  6., 15.])

In [28]:
df['zeroBalCode'] = df.zeroBalCode.replace(1,0)
df['zeroBalCode'] = df.zeroBalCode.replace([9,3,16,6,2,15],1)

In [29]:
df.zeroBalCode.unique()

array([0., 1.])

In [30]:
#Checking datatypes of individual feature
df.zeroBalCode.value_counts()

0.0    33440
1.0     1948
Name: zeroBalCode, dtype: int64

# Review values for dates and slice (if needed) the timeframe you want

In [31]:
df.origYear.unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

In [32]:
df['origMonth'].unique()

array([11, 12,  1,  2,  3,  5,  4,  6,  7,  9,  8, 10], dtype=int64)

# Coerce the column dtype (int in this case)

In [33]:
df = df.astype({'origLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'borrCreditScore':'int','mortInsType':'int','bestCreditScore':'int',\
               'worstCreditScore':'int','avgCreditScore':'int','zeroBalCode':'int'})

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35388 entries, 0 to 35387
Data columns (total 31 columns):
origChannel           35388 non-null int64
origIntRate           35388 non-null float64
origUPB               35388 non-null int64
origLTV               35388 non-null int32
numBorrowers          35388 non-null int32
origDebtIncRatio      35388 non-null int32
borrCreditScore       35388 non-null int32
loanPurp              35388 non-null int64
zipCode               35388 non-null int64
pMIperct              35388 non-null float64
mortInsType           35388 non-null int32
bestCreditScore       35388 non-null int32
worstCreditScore      35388 non-null int32
avgCreditScore        35388 non-null int32
bankNumber            35388 non-null int64
stateNumber           35388 non-null int64
mSA                   35388 non-null int64
zeroBalCode           35388 non-null int32
fmacRateMax           35388 non-null float64
fmacRateMin           35388 non-null float64
fmacRateAvg           3

In [35]:
#df.to_csv('D:\FM_FULL_EPOCH3_MLReady.csv')

# Oversample dataset

In [37]:
oversample = imblearn.over_sampling.RandomOverSampler(sampling_strategy='minority')

# Review metrics last time before run (to have record of starting values)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.zeroBalCode.value_counts()

#### Foreclosures represent 5.5% of Total Events (All mortgages considered 'resolved' with an event)

# Pycaret - Setup with categorical definition

### Use feature engineering, polynomial features, trig feats, PCA

In [ ]:
# models=setup(df, target = 'rateDiffPct',
#              feature_interaction=True, feature_ratio=True,
#              polynomial_features=True, pca=True, pca_components=10,
#             numeric_features = ['origUPB','origLTV','numBorrowers','origDebtIncRatio',\
#                                'avgCreditScore'],
#             ignore_features = ['origIntRate','rank','deal'])

### Use feature engineering, polynomial features, trig feats

In [ ]:
models=setup(df, target = 'zeroBalCode')

## Compare Models with blacklist exclusions

In [ ]:
model_results=compare_models(blacklist = ['tr','ransac']) #,'lar','par','huber','llar','lasso','en','ridge','omp','br','svm'])
model_results

## Create models - store in variable

In [ ]:
catboost =create_model('catboost', fold = 10)

In [ ]:
catboost =create_model('catboost', fold = 50)

### Catboost fold results (not shown - reran in same cell)
* 10   0.5?
* 20   0.5633
* 50   0.5633

In [ ]:
lightgbm = create_model('lightgbm', fold=50)

### LightGBM fold results (not shown - reran in same cell)
    * 10   0.5?
    * 20   0.5?
    * 50   0.5585

In [ ]:
xgboost = create_model('xgboost', fold = 50)

## Tune Models

In [ ]:
tuned_catboost = tune_model('catboost') # from fold=50

* Made no difference creating the model with fold= 10 vs. fold = 50 so use fold = 10

In [ ]:
tuned_lightgbm = tune_model('lightgbm')

In [ ]:
tuned_xgboost = tune_model('xgboost')

## Ensemble models

In [ ]:
catboost_bagged = ensemble_model(catboost)

In [ ]:
catboost_boosted = ensemble_model(catboost)

## Blend 3 chosen Models

#### First blend 'non-tuned' models

In [ ]:
blend3 = blend_models(estimator_list = [catboost, lightgbm, xgboost])

#### Next blend tuned models

In [ ]:
blendTuned3 = blend_models(estimator_list = [tuned_catboost, tuned_lightgbm, tuned_xgboost])

## Evaluate Models

In [ ]:
plot_model(lightgbm, plot = 'residuals')

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:
plot_model(lightgbm, plot='vc')

In [ ]:
plot_model(lightgbm, plot = 'feature')

In [ ]:
interpret_model(lightgbm, plot='correlation', feature='avgCreditScore')

In [ ]:
interpret_model(tuned_catboost)

In [ ]:
save_model()

In [ ]:
interpret_model(estimator=logreg_model, plot='reason')

In [ ]:
logreg_model2=create_model('xgboost')

In [ ]:
plot_model(estimator=logreg_model2, plot='feature')

In [ ]:
plot_model(estimator=logreg_model2)

In [ ]:
tunned_logreg_model=tune_model('catboost')

In [ ]:
interpret_model(estimator=tunned_logreg_model,plot='summary')

In [ ]:
plot_model(estimator=tunned_logreg_model,plot='feature')

In [ ]:
interpret_model(estimator=catboost, plot='correlation')

In [ ]:
interpret_model(estimator=catboost,plot='summary')